# Aesthetic Feature Extraction

##### By Aesthetic feature extraction we refer to the aesthetical components of an image this mainly refers to Color, Composition and Texture of an adcreative. This features are extracted on the top of the assumption the peoples are attracted to beauty as a whole eventhough different persons have different perception of beauty.

In [1]:
import os
import numpy as np
import cv2

COLOR

In [11]:
class ImageColor():
    def __init__(self, image_path):
        super(ImageColor, self).__init__()

        # Read in the image
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None

        self.h_mean, self.s_mean, self.v_mean = None, None, None
        self.h_std, self.s_std, self.v_std = None, None, None

        # Read in the image
        self.read_in(image_path)

    def update(self, image_path):
        self.read_in(image_path)

    def read_in(self, image_path):
        # This function reads the image into the bgr(blue, green, red), hsv(hue, saturation, value) and gray_scale image.

        self.bgr_img = cv2.imread(image_path)
        self.hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)
        self.gray_img = cv2.imread(image_path, 0)

        # Compute the statics
        self.h_mean, self.s_mean, self.v_mean = np.mean(self.hsv_img, axis=(0, 1))
        self.h_std, self.s_std, self.v_std = np.std(self.hsv_img, axis=(0, 1))

    @staticmethod
    def compute_circular(channel_image):
        A = np.cos(channel_image).sum()
        B = np.sin(channel_image).sum()

        R = 1 - np.sqrt(A ** 2 + B ** 2) / (channel_image.shape[0] * channel_image.shape[1])

        return R

    def compute_hsv_statics(self):
        h_circular = self.compute_circular(self.hsv_img[0])
        v_intensity = np.sqrt((self.hsv_img[-1] ** 2).mean())

        return [self.s_mean, self.s_std, self.v_std, h_circular, v_intensity]

    def compute_emotion_based(self):
        valence = 0.69 * self.v_mean + 0.22 * self.s_mean
        arousal = -0.31 * self.v_mean + 0.6 * self.s_mean
        dominance = -0.76 * self.v_mean + 0.32 * self.s_mean

        return [valence, arousal, dominance]

    def compute_valence(self):
        valence = 0.69 * self.v_mean + 0.22 * self.s_mean

        return valence

    def compute_arousal(self):
        arousal = -0.31 * self.v_mean + 0.6 * self.s_mean

        return arousal

    def compute_dominance(self):
        dominance = -0.76 * self.v_mean + 0.32 * self.s_mean

        return dominance

    def compute_color_diversity(self):
        """Adapted from
        https://github.com/yilangpeng/computational-aesthetics/blob/27ff52b47b880bd46a14a7b062a4dde69b6a9988/basic.py#L46-L56
        """
        rgb = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2RGB).astype(float)

        l_rgbR, l_rgbG, l_rgbB = cv2.split(rgb)
        l_rg = l_rgbR - l_rgbG
        l_yb = 0.5 * l_rgbR + 0.5 * l_rgbG - l_rgbB

        rg_sd = np.std(l_rg)
        rg_mean = np.mean(l_rg)
        yb_sd = np.std(l_yb)
        yb_mean = np.mean(l_yb)

        rg_yb_sd = (rg_sd ** 2 + yb_sd ** 2) ** 0.5
        rg_yb_mean = (rg_mean ** 2 + yb_mean ** 2) ** 0.5
        colorful = rg_yb_sd + (rg_yb_mean * 0.3)

        return [colorful]

    def compute_color_info(self):
        hsv_res = self.compute_hsv_statics()
        emotion_res = self.compute_emotion_based()
        color_div_res = self.compute_color_diversity()

        return hsv_res + emotion_res + color_div_res

In [3]:
image = ImageColor("/home/michael_getachew/creative-optimization/creative-optimisation-cv/data/Challenge_Data/Assets/0a59be2e7dd53d6de11a10ce3649c081/_preview.png")
res_diversity = image.compute_color_diversity()
res_emotion = image.compute_emotion_based()
res_hsvstat = image.compute_hsv_statics()
print(res_diversity)
print(res_emotion)


[27.311312694085938]
[137.71552599999998, -34.069985111111116, -129.31574844444444]


COMPOSITION

In [5]:
import functools
import operator
import numpy as np
import pywt
from sklearn.cluster import (
    MeanShift,
    estimate_bandwidth
)
import cv2

In [13]:
class Composition():
    def __init__(self, image_path):
        super(Composition, self).__init__()
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None
        self.read_in(image_path)

    def compute_edge_pixels(self,
                            blur_size: int = 3,
                            ratio_low=0.4, ratio_up=0.8):
        """Adapted from
        https://github.com/yilangpeng/computational-aesthetics/blob/master/edge.py
        """
        h, w = self.bgr_img.shape[:2]
        blur_img = cv2.GaussianBlur(self.gray_img, (blur_size, blur_size), 0)

        thresh_low = min(100, np.quantile(blur_img, q=ratio_low))
        thresh_up = max(200, np.quantile(blur_img, q=ratio_up))

        edges_img = cv2.Canny(blur_img,
                              threshold1=thresh_low,
                              threshold2=thresh_up)
        num_edges = np.count_nonzero(edges_img) / (h * w)

        return [num_edges]

    def update(self, image_path):
        self.read_in(image_path)

    def read_in(self, image_path):
        self.bgr_img = cv2.imread(image_path)
        self.hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)
        self.gray_img = cv2.imread(image_path, 0)

    def compute_level_of_details(self,
                                 quantile=0.2,
                                 n_samples=3000,
                                 thresh=0.05):
        # Using quick shift segmentation
        rgb_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2RGB)

        # Flatten the image
        flat_image = rgb_img.reshape(-1, 3)
        flat_image = np.float32(flat_image)

        bandwidth = estimate_bandwidth(flat_image,
                                       quantile=quantile,
                                       n_samples=n_samples)
        mean_shift = MeanShift(bandwidth, bin_seeding=True)
        mean_shift.fit(flat_image)
        image_labels = mean_shift.labels_

        h, w = rgb_img.shape[:2]
        unique_labels, unique_counts = np.unique(image_labels,
                                                 return_counts=True)

        # Remove small region of image
        mean_thresh = (h * w) * thresh
        unique_labels = unique_labels[unique_counts > mean_thresh]
        unique_counts = unique_counts[unique_counts > mean_thresh]

        num_seg = len(unique_labels)
        average_size = unique_counts.mean() / (h * w)

        return [num_seg, average_size]

    @staticmethod
    def compute_channel_depth_of_field(channel):
        level_wanted = channel[1]
        h, w = level_wanted[0].shape[:2]

        # Create blank image to include all channel
        blank_image = np.zeros((h, w, 3))

        for idx, level_wanted_matrix in enumerate(level_wanted):
            blank_image[..., idx] = np.abs(level_wanted_matrix)

        
        # Compute M6, M7, M10, M11
        start_x, end_x = int(h / 4), int(3 * h / 4)
        start_y, end_y = int(w / 4), int(3 * w / 4)

        dof_channel = np.sum(blank_image[start_x:end_x, start_y:end_y, :]) / np.sum(blank_image)

        return dof_channel

    def compute_depth_of_field(self):
        # Process for the H channel
        h_wavelet = pywt.wavedec2(self.hsv_img[..., 0], mode="periodization",
                                  wavelet="db3", level=3)
        h_dof = self.compute_channel_depth_of_field(h_wavelet)

        # Process for the S channel
        s_wavelet = pywt.wavedec2(self.hsv_img[..., 1], mode="periodization",
                                  wavelet="db3", level=3)
        s_dof = self.compute_channel_depth_of_field(s_wavelet)

        # Process for the V channel
        v_wavelet = pywt.wavedec2(self.hsv_img[..., 2], mode="periodization",
                                  wavelet="db3", level=3)
        v_dof = self.compute_channel_depth_of_field(v_wavelet)

        return [h_dof, s_dof, v_dof]

    def compute_h_dof(self):
        # Process for the H channel
        h_wavelet = pywt.wavedec2(self.hsv_img[..., 0], mode="periodization",
                                  wavelet="db3", level=3)
        h_dof = self.compute_channel_depth_of_field(h_wavelet)

        return h_dof

    def compute_s_dof(self):
        # Process for the S channel
        s_wavelet = pywt.wavedec2(self.hsv_img[..., 1], mode="periodization",
                                  wavelet="db3", level=3)
        s_dof = self.compute_channel_depth_of_field(s_wavelet)

        return s_dof

    def compute_v_dof(self):
        # Process for the V channel
        v_wavelet = pywt.wavedec2(self.hsv_img[..., 2], mode="periodization",
                                  wavelet="db3", level=3)
        v_dof = self.compute_channel_depth_of_field(v_wavelet)

        return v_dof

    def compute_rule_of_third(self):
        h, w = self.bgr_img.shape[:2]

        # Convert to hsv
        hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)

        # Set the end points of image
        start_h, end_h = int(h / 3), int(2 * h / 3)
        start_w, end_w = int(w / 3), int(2 * w / 3)
        center_image = hsv_img[start_h:end_h, start_w:end_w]

        # Compute the mean of saturation and value
        s_mean = np.mean(center_image[..., 1])
        v_mean = np.mean(center_image[..., 2])

        return [s_mean, v_mean]

    def compute_s_mean(self):
        h, w = self.bgr_img.shape[:2]

        # Convert to hsv
        hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)

        # Set the end points of image
        start_h, end_h = int(h / 3), int(2 * h / 3)
        start_w, end_w = int(w / 3), int(2 * w / 3)
        center_image = hsv_img[start_h:end_h, start_w:end_w]

        # Compute the mean of saturation and value
        s_mean = np.mean(center_image[..., 1])

        return s_mean

    def compute_v_mean(self):
        h, w = self.bgr_img.shape[:2]

        # Convert to hsv
        hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)

        # Set the end points of image
        start_h, end_h = int(h / 3), int(2 * h / 3)
        start_w, end_w = int(w / 3), int(2 * w / 3)
        center_image = hsv_img[start_h:end_h, start_w:end_w]

        # Compute the mean of saturation and value
        v_mean = np.mean(center_image[..., 2])
    

In [7]:
composition = Composition("/home/michael_getachew/creative-optimization/creative-optimisation-cv/data/Challenge_Data/Assets/0a22f881b77f00220f2034c21a18b854/_preview.png")
count_edges = composition.compute_depth_of_field()
edge_res = composition.compute_edge_pixels()
rule_of_thirds_res = composition.compute_rule_of_third()
print(count_edges)
print(edge_res)
print(rule_of_thirds_res)

[0.24078261539783, 0.36778763751914645, 0.33361000005577574]
[0.04909]
[0.24078261539783, 0.36778763751914645, 0.33361000005577574]
[71.72994011976049, 75.76410179640719]


Texture

In [8]:
import math
import cv2
import numpy as np
import pywt
import skimage.measure
import skimage.feature
import skimage.morphology
import skimage.filters.rank

In [9]:
class Texture():
    def __init__(self, image_path):
        super(Texture, self).__init__()
        self.bgr_img = None
        self.hsv_img = None
        self.gray_img = None
        self.read_in(image_path)

    def compute_entropy(self):
        entropy = skimage.filters.rank.entropy(image=self.gray_img,
                                               selem=skimage.morphology.square(9))
        entropy = entropy.mean()
        return [entropy]

    def update(self, image_path):
        self.read_in(image_path)

    def read_in(self, image_path):
        self.bgr_img = cv2.imread(image_path)
        self.hsv_img = cv2.cvtColor(self.bgr_img, cv2.COLOR_BGR2HSV)
        self.gray_img = cv2.imread(image_path, 0)

    @staticmethod
    def compute_channel_wavlet(wavelets):
        result_value = []
        level_mean_value = 0.0
        for wavelet in wavelets[1:]:
            wavelet_feature = 0.0
            magnitude = 0.0

            temp_level_mean_value = 0
            for level_list in wavelet:
                wavelet_feature += np.sum(level_list)
                magnitude += (level_list.shape[0] * level_list.shape[1])
                temp_level_mean_value += np.mean(level_list)

            level_mean_value += (temp_level_mean_value / 3)
            result_value.append(wavelet_feature / magnitude)

        result_value.append(level_mean_value)
        return 

    
    def coarseness(self, max_k=5):
        """Adapted from
        https://github.com/Sdhir/TamuraFeatures/blob/f22f99a5898b2414e1c3f89d464a3a761e8b6a98/Tamura.m#L26-L191
        https://github.com/MarshalLeeeeee/Tamura-In-Python/blob/1a079acce55989d65fb1a676e0bf6b9fbe54be82/tamura-numpy.py#L4-L37
        """
        gray_img = np.copy(self.gray_img)
        h, w = gray_img.shape
        if 2 ** max_k >= w or 2 ** max_k >= h:
            max_k = min(int(math.log(h) / math.log(2)), int(math.log(w) / math.log(2)))

        average_gray = np.zeros((max_k, h, w))
        for k in range(1, max_k + 1):
            for row in range(2 ** (k-1), h - 2**(k-1)):
                for col in range(2 ** (k-1), w - 2**(k-1)):
                    if len(gray_img[row - 2**(k-1):row + 2**(k-1), col - 2**(k-1):col + 2**(k-1)]) == 0:
                        assert 1 == 2
                    average_gray[k - 1, row, col] = \
                        gray_img[row - 2**(k-1):row + 2**(k-1) + 1, col - 2**(k-1):col + 2**(k-1) + 1].mean()

        expected_horizontal = np.zeros((max_k, h, w))
        expected_vertical = np.zeros((max_k, h, w))

        for k in range(1, max_k + 1):
            for row in range(2**(k-1), h - 2**(k-1)):
                for col in range(2**(k-1), w - 2**(k-1)):
                    expected_horizontal[k - 1, row, col] = \
                        np.abs(
                            average_gray[k - 1, row + 2**(k-1), col] - average_gray[k - 1, row - 2**(k-1), col])
                    expected_vertical[k - 1, row, col] = \
                        np.abs(
                            average_gray[k - 1, row, col + 2**(k-1)] - average_gray[k - 1, row, col - 2**(k-1)])

        coarseness_best = np.zeros((h, w))
        for row in range(h):
            for col in range(w):
                max_horizontal = np.max(expected_horizontal[:, row, col])
                argmax_horizontal = np.argmax(expected_horizontal[:, row, col])
                max_vertical = np.max(expected_vertical[:, row, col])
                argmax_vertical = np.argmax(expected_vertical[:, row, col])

                if max_horizontal > max_vertical:
                    max_arg_k = argmax_horizontal
                else:
                    max_arg_k = argmax_vertical

                coarseness_best[row, col] = 2 ** max_arg_k

        coarseness = coarseness_best.mean()

        return coarseness

    def contrast(self):
        """Adapted from
        https://github.com/Sdhir/TamuraFeatures/blob/f22f99a5898b2414e1c3f89d464a3a761e8b6a98/Tamura.m#L194-L206
        """
        gray_img = np.copy(self.gray_img).reshape(-1)
        average_value = np.mean(gray_img)

        base_value = gray_img - average_value
        fourth_moment = np.mean(np.power(base_value, 4))
        variance = np.mean(np.power(base_value, 2))

        alpha = fourth_moment / (variance ** 2)
        contrast_value = math.sqrt(variance) / math.pow(alpha, 0.25)

        return contrast_value

    def directionality(self):
        """Adapted from
        https://github.com/Sdhir/TamuraFeatures/blob/f22f99a5898b2414e1c3f89d464a3a761e8b6a98/Tamura.m#L209-L302
        """
        # Padding image for the filter
        gray_img = np.copy(self.gray_img).astype("int64")
        h, w = gray_img.shape[:2]

        horizontal_filter = np.array([[-1, 0, 1],
                                      [-1, 0, 1],
                                      [-1, 0, 1]])
        vertical_filter = np.array([[1, 1, 1],
                                    [0, 0, 0],
                                    [-1, -1, -1]])

        # Applying horizontal pattern filter
        delta_horizontal = cv2.filter2D(src=gray_img.astype(np.float), ddepth=-1,
                                        kernel=horizontal_filter)
        for wi in range(0, w - 1):
            delta_horizontal[0][wi] = gray_img[0][wi + 1] - gray_img[0][wi]
            delta_horizontal[h - 1][wi] = gray_img[h - 1][wi + 1] - gray_img[h - 1][wi]
        for hi in range(0, h):
            delta_horizontal[hi][0] = gray_img[hi][1] - gray_img[hi][0]
            delta_horizontal[hi][w - 1] = gray_img[hi][w - 1] - gray_img[hi][w - 2]

        # Applying vertical pattern filter
        delta_vertical = cv2.filter2D(src=gray_img.astype(np.float), ddepth=-1,
                                      kernel=vertical_filter)
        for wi in range(0, w):
            delta_vertical[0][wi] = gray_img[1][wi] - gray_img[0][wi]
            delta_vertical[h - 1][wi] = gray_img[h - 1][wi] - gray_img[h - 2][wi]
        for hi in range(0, h - 1):
            delta_vertical[hi][0] = gray_img[hi + 1][0] - gray_img[hi][0]
            delta_vertical[hi][w - 1] = gray_img[hi + 1][w - 1] - gray_img[hi][w - 1]

        delta_magnitude = (np.abs(delta_horizontal) + np.abs(delta_vertical)) / 2.0
        delta_magnitude_vec = delta_magnitude.reshape(-1)

        # Calculate the angle (theta)
        theta = np.zeros([h, w])
        for row in range(h):
            for col in range(w):
                if delta_horizontal[row][col] == 0 and delta_vertical[row][col] == 0:
                    theta[row][col] = 0
                elif delta_horizontal[row][col] == 0:
                    theta[row][col] = np.pi
                else:
                    theta[row][col] = np.arctan(delta_vertical[row][col] / delta_horizontal[row][col]) + np.pi / 2.0
        theta_vec = theta.reshape(-1)

        n = 16
        thresh = 12
        HD = np.zeros(n)

        magnitude_len = delta_magnitude_vec.shape[0]
        for ni in range(n):
            for k in range(magnitude_len):
                if (delta_magnitude_vec[k] >= thresh) and \
                   (theta_vec[k] >= (2 * ni - 1) * np.pi / (2 * n)) and \
                   (theta_vec[k] < (2 * ni + 1) * np.pi / (2 * n)):
                    HD[ni - 1] += 1
        HD = HD / np.sum(HD)

        directionality = 0.0
        HD_max_index = np.argmax(HD)
        for ni in range(n):
            directionality += np.power((ni - HD_max_index), 2) * HD[ni]

        return directionality

    def compute_tamura(self):
        coarseness = self.coarseness()
        contrast = self.contrast()
        directionality = self.directionality()

        return [coarseness, contrast, directionality]    
    

In [10]:
texture = Texture("/home/michael_getachew/creative-optimization/creative-optimisation-cv/data/Challenge_Data/Assets/0a22f881b77f00220f2034c21a18b854/_preview.png")
entropy_res = texture.compute_entropy()
tamura_res = texture.compute_tamura()
print(entropy_res)
print(tamura_res)

/tmp/ipykernel_263471/3475560424.py:10: FutureWarning: `selem` is a deprecated argument name for `entropy`. It will be removed in version 1.0. Please use `footprint` instead.
  entropy = skimage.filters.rank.entropy(image=self.gray_img,
/tmp/ipykernel_263471/3475560424.py:127: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  delta_horizontal = cv2.filter2D(src=gray_img.astype(np.float), ddepth=-1,
/tmp/ipykernel_263471/3475560424.py:137: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, us

[1.1893659603262525]
[10.84144, 108.4337695180842, 18.400310366232155]


Extraction of Aesthetic Features

In [15]:
import cv2
import os
import glob
import pandas as pd
from pathlib import Path
import logging

class ExtractorPipeline():
    """
    performs feature extraction from all image files in the assets folder
    Creates a directory where extracted features will be saved as CSV files
    Extracted features include: Logo, CTA button, engagement button, objects, facial features, dominant colours, texts
    Parameters: full path to the assets folder
    """

    def __init__(self, data_folder) -> None:
        self.assets_folder = data_folder
        assets_dir_path = os.path.dirname(self.assets_folder)
        self.extracted_path = str(
            Path(assets_dir_path).parent)+"/Assets"

        if not os.path.isdir(self.extracted_path):
            os.makedirs(self.extracted_path)

    def aesthetic_extractor(self):
        """
        extract the location of the logo from all preview images in the assets folder
        """
        folder_list = glob.glob(self.assets_folder)

        aesthetic_featues = []

        for folder in folder_list:
            # access folders in the assets directory
            query_img = os.path.join(folder, '_preview.png')

            # check if files exist
            if os.path.exists(query_img):
                
                #COLOR
                image = ImageColor(query_img)
                Ad_color_diversity = image.compute_color_diversity()
                Ad_color_valence = image.compute_valence()
                Ad_color_arousal = image.compute_arousal()
                Ad_color_dominace = image.compute_dominance()
                
                #COMPOSITION
                composition = Composition(query_img)
                Ad_hue_dof = composition.compute_h_dof()
                Ad_saturation_dof = composition.compute_s_dof()
                Ad_value_dof = composition.compute_v_dof()
                Ad_edge_pixel = composition.compute_edge_pixels()
                Ad_s_mean = composition.compute_s_mean()
                Ad_v_mean = composition.compute_v_mean()
                
                #TEXTURE
                texture = Texture(query_img)
                Ad_entropy = texture.compute_entropy()
                Ad_coarseness = texture.coarseness()
                Ad_directionality = texture.directionality()
                Ad_contrast = texture.contrast()
                
                aesthetic_featues.append([folder.split('/')[-1], Ad_color_diversity,Ad_color_valence,Ad_color_arousal,Ad_color_dominace,
                Ad_hue_dof,Ad_saturation_dof,Ad_value_dof,Ad_edge_pixel,Ad_s_mean,Ad_v_mean,Ad_entropy, Ad_coarseness, Ad_directionality, Ad_contrast])
            else:
                # if image does not exist
                aesthetic_featues.append([folder.split('/')[-1], 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

        # save the list elements as a dataframe
        df = pd.DataFrame(aesthetic_featues, columns=['Ad_color_diversity','Ad_color_valence','Ad_color_arousal','Ad_color_dominace',
                'Ad_hue_dof','Ad_saturation_dof','Ad_value_dof','Ad_edge_pixel','Ad_s_mean','Ad_v_mean','Ad_entropy','Ad_coarseness','Ad_directionality','Ad_contrast'])

        # save dataframe as csv file
        df.to_csv(self.extracted_path+'/aesthetic_featues.csv', index=False)

In [16]:
ExtractorPipeline("/home/michael_getachew/creative-optimization/creative-optimisation-cv/data/Challenge_Data")